In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium
from folium.raster_layers import WmsTileLayer
from folium.raster_layers import TileLayer
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
from branca.element import Figure

In [6]:
#KMeans OTB
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
#Plot geography
! pip install geopy
! pip install shapely
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [8]:
import folium
import pandas as pd
import numpy as np
from folium.raster_layers import WmsTileLayer
from folium.raster_layers import TileLayer
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
#from folium.plugins import TimeSliderChoropleth
from branca.element import Figure

In [9]:
df=pd.read_csv('datos_rios_embalses_tiempo.csv',index_col=[0])
df

,id,superficie,fecha_x,lat,lng,idprovincia,provincia_x,idmunicipio,municipio,causa,...,prec_acu,hum_med,hum_max,hum_min,v_viento_med,presion,lluvia,helada,dist_est_x,dist_est_y
0,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,...,97.0,78.0,92.0,58.0,3.0,1015.0,13.0,0.0,29.186920,29.186920
11,2001150088,1.50,2001-02-24 00:00:00-05:00,42.936918,-9.114350,15,A CORUÑA,34,DUMBRÍA,4,...,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0,34.817885,34.817885
15,2001150090,3.00,2001-02-25 00:00:00-05:00,42.643031,-8.939252,15,A CORUÑA,67,"POBRA DO CARAMIÑAL, A",4,...,98.0,76.0,92.0,59.0,5.0,1017.0,10.0,0.0,12.204302,12.204302
26,2001150094,1.50,2001-02-25 00:00:00-05:00,43.186836,-8.685470,15,A CORUÑA,19,CARBALLO,4,...,12.0,92.0,99.0,74.0,NaN,NaN,17.0,8.0,3.411028,3.411028
32,2001150111,3.80,2001-02-25 00:00:00-05:00,42.917476,-9.082862,15,A CORUÑA,45,MAZARICOS,4,...,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0,31.468092,31.468092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357650,2015360747,1.45,2015-09-06 00:00:00-04:00,42.103959,-8.522512,36,PONTEVEDRA,50,SALVATERRA DE MIÑO,4,...,108.0,76.0,96.0,48.0,3.0,NaN,10.0,0.0,5.065956,5.065956
357693,2015360751,1.00,2015-09-06 00:00:00-04:00,42.157695,-8.397080,36,PONTEVEDRA,34,"NEVES, AS",4,...,90.0,68.0,88.0,40.0,2.0,999.0,10.0,0.0,5.750805,5.750805
357743,2015360770,5.94,2015-09-08 00:00:00-04:00,42.554722,-8.018586,36,PONTEVEDRA,16,DOZÓN,4,...,117.0,76.0,90.0,55.0,19.0,905.0,9.0,0.0,7.535815,7.535815
357778,2015360794,3.20,2015-09-28 00:00:00-04:00,42.243441,-8.322375,36,PONTEVEDRA,13,COVELO,4,...,137.0,73.0,89.0,52.0,4.0,975.0,6.0,0.0,9.613333,9.613333


In [10]:
#Función distancia
from math import radians, cos, sin, asin, sqrt 
def distance(lat1,lat2,lon1,lon2): 
      
    # Convertimos los grados a radianes. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Fórmula Haversine   
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
    r = 6371

    return(c * r) 

In [11]:
# Función asignar casos a cada centroide 
def assignment(df, centroids):
    df_assig=df
    for i in centroids.keys():
        df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
        
    centroid_distance_cols = ['distance_from_{}'.format(i) for i in centroids.keys()]
    df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
    df_assig['closest'] = df_assig['closest'].map(lambda x: int(x.lstrip('distance_from_')))
    return df_assig

# Función actualizar los centroides en función de la asignación anterior
def update(df_up,centroids):
    for i in centroids.keys():
        centroids[i][0] = np.mean(df_up[df_up['closest'] == i]['lat'])
        centroids[i][1] = np.mean(df_up[df_up['closest'] == i]['lng'])
    return centroids

In [12]:
#Función Kmeans Custom
import copy
import random 
def KMeans_geo(df,k,iter_max=500,cinit_meth='sample_point'):
    df_KMeans_geo=df
    # centroids[i] = [lat, lng]
    if cinit_meth == 'rectangular':
        lat_min= df_KMeans_geo['lat'].min()
        lat_max= df_KMeans_geo['lat'].max()
        lng_min= df_KMeans_geo['lng'].min()
        lng_max= df_KMeans_geo['lng'].max()
        centroids = {
            i+1: [lat_min+(lat_max-lat_min)*random.random(), lng_min+(lng_max-lng_min)*random.random()]
            for i in range(k)
        }
    else:
        x = {i+1: np.random.randint(0, len(df_KMeans_geo)-1) for i in range(k)}
        centroids = { 
            j+1: [df_KMeans_geo['lat'][df_KMeans_geo.index[x[j+1]]], df_KMeans_geo['lng'][df_KMeans_geo.index[x[j+1]]]]
            for j in range(k)
        }
    df_KMeans_geo = assignment(df_KMeans_geo, centroids)
    j = 1
    while True:
        centroids_old = df_KMeans_geo['closest'].copy(deep=True)
        centroids = update(df_KMeans_geo,centroids)
        df_KMeans_geo = assignment(df_KMeans_geo, centroids)
        centroids_new = df_KMeans_geo['closest'].copy(deep=True)
        j+=1
        if centroids_old.equals(centroids_new):
            break
        if iter_max == j:
            break
    return(df_KMeans_geo,centroids,j)    


In [13]:
df_coord = df[['id','lat','lng']]
df_coord

,id,lat,lng
0,2001150021,43.703581,-8.038777
11,2001150088,42.936918,-9.114350
15,2001150090,42.643031,-8.939252
26,2001150094,43.186836,-8.685470
32,2001150111,42.917476,-9.082862
...,...,...,...
357650,2015360747,42.103959,-8.522512
357693,2015360751,42.157695,-8.397080
357743,2015360770,42.554722,-8.018586
357778,2015360794,42.243441,-8.322375


In [ ]:
df_KMeans_Loop={}
for i in range(40,51):
    print('Empezando iteracción '+str(i))
    df_coord = df[['id','lat','lng']]
    df_KMeans_iter= KMeans_geo(df_coord,i)
    df_KMeans_Loop[format(i)]=df_KMeans_iter
    

Empezando iteracción 40


<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

...

<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

Empezando iteracción 44


<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

<ipython-input-11-a0fdbae447c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['distance_from_{}'.format(i)] = df_assig.apply( lambda x: distance(x['lat'],centroids[i][0],x['lng'],centroids[i][1]), axis=1)
<ipython-input-11-a0fdbae447c5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assig['closest'] = df_assig.loc[:, centroid_distance_cols].idxmin(axis=1)
<ipython-input-11-a0fdbae447c5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [13]:
np.save('Ktorres.npy', df_KMeans_Loop) 

In [16]:
# Carga
KMeans_torres = np.load('Ktorres.npy',allow_pickle='TRUE').item()
KMeans_torres[str(40)]

(                id        lat       lng  distance_from_1  distance_from_2  \
 0       2001150021  43.703581 -8.038777       171.684290       120.150046   
 11      2001150088  42.936918 -9.114350       128.553327       133.947441   
 15      2001150090  42.643031 -8.939252        97.250387       117.368644   
 26      2001150094  43.186836 -8.685470       128.911119       110.507845   
 32      2001150111  42.917476 -9.082862       125.201184       131.027653   
 ...            ...        ...       ...              ...              ...   
 357662  2015360747  42.103959 -8.522512        47.548434       106.297567   
 357705  2015360751  42.157695 -8.397080        36.767481        94.500059   
 357755  2015360770  42.554722 -8.018586        44.138733        44.801787   
 357790  2015360794  42.243441 -8.322375        31.933886        83.756162   
 357821  2015360819  42.658303 -8.268949        61.155338        62.550329   
 
         distance_from_3  distance_from_4  distance_from_5  di

In [17]:
Torres=pd.DataFrame(columns=['K','id_torre','lat','lng'])
for k in range(40,51):
     for i in range(1,k+1):
        Torres=Torres.append({'K':str(k),
                              'id_torre':i,
                              'lat':KMeans_torres[str(k)][1][i][0],
                              'lng':KMeans_torres[str(k)][1][i][1]}, ignore_index=True)
Torres

,K,id_torre,lat,lng
0,40,1,42.160924,-7.951038
1,40,2,42.695401,-7.505463
2,40,3,41.895499,-7.485817
3,40,4,42.349367,-7.535447
4,40,5,42.302216,-7.172313
...,...,...,...,...
490,50,46,42.405896,-8.718643
491,50,47,42.626724,-8.675384
492,50,48,NaN,NaN
493,50,49,42.272706,-7.186541


In [18]:
inc_clasificados=pd.DataFrame(columns=['K','id_inc','lat','lng','id_torre','dist_torre'])
l=len(df)
for k in range(40,51):
     print('K:'+str(k))
     for i in KMeans_torres[str(k)][0].index:
        inc_clasificados=inc_clasificados.append({'K':str(k),
                              'id_inc':KMeans_torres[str(k)][0]['id'][i],
                              'lat':KMeans_torres[str(k)][0]['lat'][i],
                              'lng':KMeans_torres[str(k)][0]['lng'][i],
                              'id_torre':KMeans_torres[str(k)][0]['closest'][i],
                              'dist_torre':KMeans_torres[str(k)][0]['distance_from_'+str(KMeans_torres[str(k)][0]['closest'][i])][i]}, ignore_index=True)
inc_clasificados     

K:40
K:41
K:42
K:43
K:44
K:45
K:46
K:47
K:48
K:49
K:50


,K,id_inc,lat,lng,id_torre,dist_torre
0,40,2001150021,43.703581,-8.038777,13,18.225231
1,40,2001150088,42.936918,-9.114350,38,6.794885
2,40,2001150090,42.643031,-8.939252,25,2.989116
3,40,2001150094,43.186836,-8.685470,26,11.322706
4,40,2001150111,42.917476,-9.082862,38,9.154385
...,...,...,...,...,...,...
270430,50,2015360747,42.103959,-8.522512,37,4.082425
270431,50,2015360751,42.157695,-8.397080,9,6.714911
270432,50,2015360770,42.554722,-8.018586,25,4.427249
270433,50,2015360794,42.243441,-8.322375,9,7.180978


In [31]:
#inc_clasificados.to_csv('inc_clasificados.csv')

In [67]:
torres_javi = pd.merge(Torres,inc_clasificados, how='left', on=['K','id_torre'])
torres_javi = torres_javi.rename(columns={'lat_x' : 'lat_torre',
                                         'lng_x': 'lng_torre',
                                         'lat_y':'lat_inc',
                                         'lng_y' : 'lng_inc'})
torres_javi

,K,id_torre,lat_x,lng_x,id_inc,lat_y,lng_y,dist_torre
0,40,1,42.160924,-7.951038,2001320089,42.167965,-7.981960,2.666112
1,40,1,42.160924,-7.951038,2001320137,42.114404,-8.027610,8.162121
2,40,1,42.160924,-7.951038,2001320144,42.213530,-8.070287,11.434443
3,40,1,42.160924,-7.951038,2001320222,42.193373,-7.871413,7.488074
4,40,1,42.160924,-7.951038,2001320245,42.217713,-7.879339,8.646941
...,...,...,...,...,...,...,...,...
270438,50,50,43.250032,-7.473288,2015270400,43.377018,-7.199805,26.248134
270439,50,50,43.250032,-7.473288,2015270416,43.204373,-7.303771,14.642862
270440,50,50,43.250032,-7.473288,2015270531,43.516043,-7.218887,36.022254
270441,50,50,43.250032,-7.473288,2015270557,43.359768,-7.177782,26.845222


In [171]:
torres_javi.to_csv('torres_javi.csv')

In [70]:
torres_distancia=torres_javi.groupby(['K','id_torre'])['dist_torre'].max()
torres_dist_max = pd.merge(torres_javi, torres_distancia, on=['K','id_torre'],how='left')
torres_dist_max=torres_dist_max.rename(columns={'dist_torre_x' : 'distancia_a_torre',
                                'dist_torre_y' : 'radio_torre'})
torres_dist_max

K   id_torre
40  1           17.103859
    2           28.704195
    3           12.482996
    4           19.464920
    5           15.768069
                  ...    
50  46          18.447641
    47          15.705601
    48                NaN
    49          15.245396
    50          46.245926
Name: dist_torre, Length: 495, dtype: float64

Una vez obtenida la distancia máxima de los incendios correspondientes a una torre, guardamos las localizaciones de la torre, junto esta distancia que será su radio de control

In [87]:
torres_unicas_dist=torres_dist_max[['K','id_torre','lat_torre','lng_torre','radio_torre']].drop_duplicates()
torres_unicas_dist

,K,id_torre,lat_torre,lng_torre,radio_torre
0,40,1,42.160924,-7.951038,17.103859
719,40,2,42.695401,-7.505463,28.704195
1304,40,3,41.895499,-7.485817,12.482996
1688,40,4,42.349367,-7.535447,19.464920
2185,40,5,42.302216,-7.172313,15.768069
...,...,...,...,...,...
268536,50,46,42.405896,-8.718643,18.447641
268756,50,47,42.626724,-8.675384,15.705601
269102,50,48,NaN,NaN,NaN
269103,50,49,42.272706,-7.186541,15.245396


In [170]:
torres_unicas_dist.to_csv('torres_unicas_dist.csv')

In [90]:
torres_unicas_dist=torres_unicas_dist.reset_index()
torres_unicas_dist=torres_unicas_dist.reset_index()

## Generación del gráfico de KMeans

In [141]:
for k in range(40,51):
    print( str(k) +'Torres')
    #inicialización gráfico
    EsriImagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    EsriAttribution = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"
    figure2=folium.Figure(width=550,height=550)
    MapCentroids = folium.Map(location=[42.733581, -7.838777], tiles=EsriImagery, attr=EsriAttribution, zoom_start=8)

    m = figure2.add_child(MapCentroids)
    for j in range(1,k+1):
        print('Torre'+str(j))
        if ~torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['radio_torre'].isna().values[0]:
            popup = """
            Torre : <b>%s</b><br>
            Radio de control : <b>%s</b><br>
            """ % (j,df_k[t])
            folium.Circle(
                location = [torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lat_torre'],
                            torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lng_torre']],
                radius = torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['radio_torre'].values[0]*1000,
                weight = 0,
                color = 'orange',
                fill_color = 'orange',
                fill_opacity = 0.3,
                fill = True).add_to(MapCentroids)
            folium.Marker(
                location=[torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lat_torre'],
                          torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lng_torre']],
                popup = popup,
                icon=folium.Icon(color='orange',prefix='glyphicon',icon='fire')).add_to(MapCentroids)
        else:
            None
    MapCentroids.save(str(k) +'Torres.html')


40Torres
Torre1
Torre2
Torre3
Torre4
Torre5
Torre6
Torre7
Torre8
Torre9
Torre10
Torre11
Torre12
Torre13
Torre14
Torre15
Torre16
Torre17
Torre18
Torre19
Torre20
Torre21
Torre22
Torre23
Torre24
Torre25
Torre26
Torre27
Torre28
Torre29
Torre30
Torre31
Torre32
Torre33
Torre34
Torre35
Torre36
Torre37
Torre38
Torre39
Torre40
41Torres
Torre1
Torre2
Torre3
Torre4
Torre5
Torre6
Torre7
Torre8
Torre9
Torre10
Torre11
Torre12
Torre13
Torre14
Torre15
Torre16
Torre17
Torre18
Torre19
Torre20
Torre21
Torre22
Torre23
Torre24
Torre25
Torre26
Torre27
Torre28
Torre29
Torre30
Torre31
Torre32
Torre33
Torre34
Torre35
Torre36
Torre37
Torre38
Torre39
Torre40
Torre41
42Torres
Torre1
Torre2
Torre3
Torre4
Torre5
Torre6
Torre7
Torre8
Torre9
Torre10
Torre11
Torre12
Torre13
Torre14
Torre15
Torre16
Torre17
Torre18
Torre19
Torre20
Torre21
Torre22
Torre23
Torre24
Torre25
Torre26
Torre27
Torre28
Torre29
Torre30
Torre31
Torre32
Torre33
Torre34
Torre35
Torre36
Torre37
Torre38
Torre39
Torre40
Torre41
Torre42
43Torres
Torre1


In [143]:
MapCentroids


In [209]:
k=50
for i in range(1,len(inc_clasificados[(inc_clasificados['K']==str(k)) & (inc_clasificados['id_torre']==18)])):
    folium.Circle(
        location = [inc_clasificados[(inc_clasificados['K']==str(k)) & (inc_clasificados['id_torre']==18)].reset_index()['lat'][i-1],
                    inc_clasificados[(inc_clasificados['K']==str(k)) & (inc_clasificados['id_torre']==18)].reset_index()['lng'][i-1]],
        radius = 100,
        color = 'red',
        fill_color = 'red',
        fill_opacity = 1,
        fill = True).add_to(MapCentroids)

In [208]:
m

In [176]:
def map_k(k):
    #inicialización gráfico
    EsriImagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    EsriAttribution = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"
    figure2=folium.Figure(width=550,height=550)
    MapCentroids = folium.Map(location=[42.733581, -7.838777], tiles=EsriImagery, attr=EsriAttribution, zoom_start=8)

    m = figure2.add_child(MapCentroids)
    for j in range(1,k+1):
        if ~torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['radio_torre'].isna().values[0]:
            popup = """
            Torre : <b>%s</b><br>
            Radio de control : <b>%s</b><br>
            """ % (j,df_k[t])
            folium.Circle(
                location = [torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lat_torre'],
                            torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lng_torre']],
                radius = torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['radio_torre'].values[0]*1000,
                weight = 0,
                color = 'orange',
                fill_color = 'orange',
                fill_opacity = 0.3,
                fill = True).add_to(MapCentroids)
            folium.Marker(
                location=[torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lat_torre'],
                          torres_unicas_dist[(torres_unicas_dist['K']==str(k)) & (torres_unicas_dist['id_torre']==j)]['lng_torre']],
                popup = popup,
                icon=folium.Icon(color='orange',prefix='glyphicon',icon='fire')).add_to(MapCentroids)
        else:
            None
    return MapCentroids

In [177]:
map_k(44)